In [12]:
library(tidyverse)
library(repr)
library(readxl)

data <- read_delim("processed.switzerland.csv", delim = ",", col_names = FALSE) |>
    rename(age = X1, sex = X2, cp = X3, trestbps = X4, chol = X5, fbs = X6, restecg = X7, thalach = X8, exang = X9, 
           oldpeak = X10, slope = X11, ca = X12, thal = X13, num_predicted = X14)

#dataframe be used
heart_data <- data |>
    select(age, sex, cp, trestbps, chol, num_predicted)
heart_data 

Rows: 123 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (9): X4, X6, X7, X8, X9, X10, X11, X12, X13
dbl (5): X1, X2, X3, X5, X14

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


age,sex,cp,trestbps,chol,num_predicted
<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
32,1,1,95,0,1
34,1,4,115,0,1
35,1,4,?,0,3
36,1,4,110,0,1
38,0,4,105,0,2
38,0,4,110,0,1
38,1,3,100,0,0
38,1,3,115,0,1
38,1,4,135,0,2
